In [8]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from torchvision.utils import save_image
import os

In [ ]:
# Transform to resize images and normalize them
transform = transforms.Compose([
    transforms.Resize(64),  # Resize images to 64x64
    transforms.CenterCrop(64),  # Crop the image to 64x64
    transforms.ToTensor(),  # Convert images to tensor
    transforms.Normalize([0.5], [0.5])  # Normalize to [-1, 1]])

In [ ]:
# Create the directory if it doesn't exist
os.makedirs('generated_images', exist_ok=True)

# Load your dataset (replace 'crack_images' with the actual directory of your dataset)
dataset = datasets.ImageFolder(root="crack_dataset", transform=transform)

# Create DataLoader for training
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4)

In [ ]:
# DCGAN Generator Network
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # Input is Z, going into a fully connected layer
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            # State size. (512) x 4 x 4
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            # State size. (256) x 8 x 8
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            # State size. (128) x 16 x 16
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            # State size. (64) x 32 x 32
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()  # Final output layer, scale the output to [-1, 1] for image pixel values
            # State size. (3) x 64 x 64
        )

    def forward(self, x):
        return self.main(x)

In [ ]:

# DCGAN Discriminator Network
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # State size. (64) x 32 x 32
            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            # State size. (128) x 16 x 16
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            # State size. (256) x 8 x 8
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            # State size. (512) x 4 x 4
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()  # Output probability
        )

    def forward(self, x):
        return self.main(x)

In [ ]:
# Initialize the models
generator = Generator()
discriminator = Discriminator()

# Loss function and optimizers
criterion = nn.BCELoss()
optimizer_g = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [ ]:

# Training loop
num_epochs = 25
for epoch in range(num_epochs):
    for i, (data, _) in enumerate(dataloader):
        # Create labels for the real and fake images
        real_images = data
        batch_size = real_images.size(0)
        labels_real = torch.ones(batch_size, 1)
        labels_fake = torch.zeros(batch_size, 1)

      # Train the Discriminator
        discriminator.zero_grad()

        # Get discriminator's output for real images
        output_real = discriminator(real_images)
        output_real = output_real.view(-1)  # Squeeze to shape [batch_size]
        labels_real = labels_real.view(-1)  # Ensure labels are also [batch_size]
        loss_d_real = criterion(output_real, labels_real)

        # Create fake images
        noise = torch.randn(batch_size, 100, 1, 1)
        fake_images = generator(noise)

        # Get discriminator's output for fake images
        output_fake = discriminator(fake_images.detach())
        output_fake = output_fake.view(-1)  # Squeeze to shape [batch_size]
        labels_fake = labels_fake.view(-1)  # Ensure labels are also [batch_size]
        loss_d_fake = criterion(output_fake, labels_fake)

        # Total discriminator loss
        loss_d = loss_d_real + loss_d_fake
        loss_d.backward()
        optimizer_d.step()



        # Train the Generator
        generator.zero_grad()

        # Get discriminator's output for fake images
        output_fake = discriminator(fake_images)
        output_fake = output_fake.view(-1)  # Squeeze to shape [batch_size]

        # Calculate generator loss
        loss_g = criterion(output_fake, labels_real)  # labels_real is already [batch_size]
        loss_g.backward()
        optimizer_g.step()


        # Print the losses
        if i % 50 == 0:
            print(f"Epoch [{epoch}/{num_epochs}], Step [{i}/{len(dataloader)}], "
                  f"D Loss: {loss_d.item()}, G Loss: {loss_g.item()}")

    # Save generated images every epoch
    save_image(fake_images.data, f"generated_images/epoch_{epoch}.png", normalize=True)



Epoch [0/25], Step [0/8], D Loss: 1.2859406471252441, G Loss: 2.8667752742767334
Epoch [1/25], Step [0/8], D Loss: 0.37672871351242065, G Loss: 7.609241008758545
Epoch [2/25], Step [0/8], D Loss: 0.14128120243549347, G Loss: 10.725042343139648
Epoch [3/25], Step [0/8], D Loss: 0.11249386519193649, G Loss: 15.652231216430664
Epoch [4/25], Step [0/8], D Loss: 2.3848040103912354, G Loss: 16.215341567993164
Epoch [5/25], Step [0/8], D Loss: 0.038313861936330795, G Loss: 8.704952239990234
Epoch [6/25], Step [0/8], D Loss: 0.05956666171550751, G Loss: 9.552754402160645
Epoch [7/25], Step [0/8], D Loss: 0.10948708653450012, G Loss: 11.875425338745117
Epoch [8/25], Step [0/8], D Loss: 0.7137685418128967, G Loss: 14.383546829223633
Epoch [9/25], Step [0/8], D Loss: 0.08276932686567307, G Loss: 6.741725921630859
Epoch [10/25], Step [0/8], D Loss: 0.4594111442565918, G Loss: 9.40829849243164
Epoch [11/25], Step [0/8], D Loss: 1.150230050086975, G Loss: 4.85722017288208
Epoch [12/25], Step [0/8], 

In [ ]:

# Save the trained models
torch.save(generator.state_dict(), "generator.pth")
torch.save(discriminator.state_dict(), "discriminator.pth")